# Exercice 1 – Nettoyage du texte médical

Nettoyer un texte médical brut en supprimant :
- Les dates (`12/03/25`)
- Les valeurs numériques avec unités (`125mg/L`)
- Les caractères spéciaux (`#`, `°`)
Et normaliser :
- Abréviations (`Ttt` → `traitement`)
- Erreurs (`pat.` → `patient`)

In [1]:
import re

In [2]:
def nettoyer_texte(texte):
    # 1. Dictionnaire d'abréviations médicales
    abreviations = {
        r"\bTtt\b": "traitement",
        r"\bpat\.?\b": "patient",
        r"\bIV\b": "intraveineuse",
        r"\bBID\b": "bis in die"
    }

    # 2. Sauvegarder les concepts composés (COVID-19, etc.)
    concepts = re.findall(r'\b\w+-\w+\b', texte)
    for i, c in enumerate(concepts):
        texte = texte.replace(c, f"__CONCEPT{i}__")
    
    # 3. Supprimer les dates
    texte = re.sub(r"\b\d{1,2}/\d{1,2}/\d{2,4}\b", "", texte)

    # 4. Supprimer les valeurs numériques avec unités (ex: 500mg/J, 39.2°C)
    texte = re.sub(r"\d+(?:[\.,]\d+)?\s*[a-zA-Z/%°]+", "", texte)

    # 5. Remplacer les abréviations
    for pattern, remplacement in abreviations.items():
        texte = re.sub(pattern, remplacement, texte, flags=re.IGNORECASE)

    # 6. Supprimer caractères spéciaux (garder tirets pour concepts médicaux)
    texte = re.sub(r"[^\w\sàâäéèêëîïôöùûüç-]", "", texte)

    # 7. Supprimer les nombres isolés (entiers ou décimaux)
    texte = re.sub(r"\b\d+(?:[\.,]\d+)?\b", "", texte)

    # 8. Supprimer les <, > ou autres symboles isolés
    texte = re.sub(r"[<>]", "", texte)

    # 9. Nettoyer les espaces multiples
    texte = re.sub(r"\s+", " ", texte)

    # 10. Restaurer les concepts médicaux protégés
    for i, c in enumerate(concepts):
        texte = texte.replace(f"__CONCEPT{i}__", c)

    return texte.strip()


In [3]:
# Test
texte_medical = """
Pat. 45ans admis le 12/03/25 pour COVID-19 sévère.
CRP: 125mg/L (N<5) - Fièvre à 39.2°C.
Ttt par corticoïdes IV + azithromycine 500mg/J.
#Attention: allergie à la pénicilline!
"""

print("Avant nettoyage:\n", texte_medical)
print("\nAprès nettoyage:\n", nettoyer_texte(texte_medical))

Avant nettoyage:
 
Pat. 45ans admis le 12/03/25 pour COVID-19 sévère.
CRP: 125mg/L (N<5) - Fièvre à 39.2°C.
Ttt par corticoïdes IV + azithromycine 500mg/J.
#Attention: allergie à la pénicilline!


Après nettoyage:
 patient admis le pour COVID-19 sévère CRP N5 - Fièvre à traitement par corticoïdes intraveineuse azithromycine Attention allergie à la pénicilline



**Interprétation :**

- Avant nettoyage, le texte représentait un extrait brut d’un compte-rendu hospitalier, typique du langage médical abrégé et codifié. Il incluait des **informations temporelles** comme la date d’admission (*12/03/25*), des **valeurs biologiques** avec unités (*CRP: 125mg/L*, *Fièvre à 39.2°C*), ainsi que des **abréviations médicales** courantes telles que *Pat.* pour *patient*, *Ttt* pour *traitement*, et *IV* pour *intraveineuse*. Le texte contenait aussi des **caractères spéciaux** comme `#`, `:` et `°`, souvent utilisés dans des notes rapides.

- Après nettoyage, le texte a été **structuré et simplifié** pour un usage automatique. Les **dates, unités et chiffres** ont été supprimés, car ils ne sont pas toujours exploitables sans contexte. Les **abréviations** ont été **normalisées** en langage courant (*Ttt* → *traitement*, *IV* → *intraveineuse*), et les **concepts médicaux importants comme "COVID-19"** ont été préservés dans leur forme originale. Les caractères parasites ont disparu, et le texte est devenu **plus lisible et prêt à être traité par des algorithmes de NLP**, tout en gardant les éléments cliniques essentiels tels que *fièvre*, *traitement*, *corticoïdes*, et *allergie à la pénicilline*.


# Exercice 2 – Tokenisation médicale

1. Conserver les concepts composés (ex: `"COVID-19"`)
2. Séparer les doses et unités (ex: `"500mg/J"` → `["500", "mg", "/", "J"]`)
3. Utiliser un dictionnaire d’abréviations

In [4]:
import nltk, re
from nltk.tokenize import word_tokenize
nltk.download('punkt')

def tokeniser(txt: str):
    # 1) Protéger les composés avec tiret (COVID-19)
    txt = re.sub(r'(\w+)-(\d+)', lambda m: m.group(0).replace('-', '§'), txt)

    # 2) Séparer chiffres et unités (ajout de 'ng' et autres unités)
    txt = re.sub(r'(\d+)(m?g|ml|µg|ng|UI|[a-zA-Z]{1,3})', r'\1 \2', txt, flags=re.IGNORECASE)
    
    # 3) Gérer les ratios avec slash (ajout de '+' pour les unités composées)
    txt = re.sub(r'([a-zA-Z]+)/([a-zA-Z]{1,3})', r'\1 / \2', txt)
    
    # 4) Tokenisation standard
    tokens = word_tokenize(txt, language='french')
    
    # 5) Restaurer les tirets protégés
    return [t.replace('§', '-') for t in tokens]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ideapad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
#pour texte non cleaning
tokeniser(texte_medical)

['Pat',
 '.',
 '45',
 'ans',
 'admis',
 'le',
 '12/03/25',
 'pour',
 'COVID-19',
 'sévère',
 '.',
 'CRP',
 ':',
 '125',
 'mg',
 '/',
 'L',
 '(',
 'N',
 '<',
 '5',
 ')',
 '-',
 'Fièvre',
 'à',
 '39.2°C',
 '.',
 'Ttt',
 'par',
 'corticoïdes',
 'IV',
 '+',
 'azithromycine',
 '500',
 'mg',
 '/',
 'J',
 '.',
 '#',
 'Attention',
 ':',
 'allergie',
 'à',
 'la',
 'pénicilline',
 '!']


Après tokenisation, le texte médical brut a été transformé en une liste de mots et de symboles isolés, ce qui permet une lecture fine et un traitement automatique efficace. Chaque élément du texte, qu’il s’agisse de mots, de chiffres, de ponctuation ou de symboles médicaux, a été séparé en unités distinctes appelées *tokens*. Les concepts médicaux composés comme **"COVID-19"** ont été correctement conservés en un seul bloc, ce qui est essentiel pour ne pas altérer le sens clinique. Les doses et unités telles que **"125mg/L"** ou **"500mg/J"** ont été finement découpées en sous-éléments (`['125', 'mg', '/', 'L']`), facilitant ainsi leur traitement ou suppression si nécessaire. Les abréviations médicales comme **"Pat"**, **"Ttt"** et **"IV"** ont été isolées, prêtes à être normalisées dans l’étape suivante. La ponctuation et les symboles spéciaux comme **"."**, **":"**, **"#"**, ou **"!"** ont également été extraits individuellement, ce qui permet un nettoyage ciblé. Cette tokenisation fine prépare le texte à un traitement plus avancé, comme la normalisation lexicale ou l’analyse sémantique, tout en conservant la structure informative du compte-rendu médical initial.



# Dictionnaire des abréviations

## Objectif
Créer un système de remplacement automatique des abréviations médicales courantes dans le texte.

In [6]:
#Créer le dictionnaire
ABREVIATIONS = {
    "ttt": "traitement",
    "BID": "bis in die",       # Signifie "deux fois par jour" en latin
    "IV": "intraveineuse",     # Administration par voie veineuse
    "TDM": "tomodensitométrie",
    "PCR": "réaction en chaîne par polymérase",
    "ECG": "électrocardiogramme"
}

In [7]:
#Tester avec un cas réel
texte_nettoye=nettoyer_texte(texte_medical)
texte_nettoye

'patient admis le pour COVID-19 sévère CRP N5 - Fièvre à traitement par corticoïdes intraveineuse azithromycine Attention allergie à la pénicilline'

In [8]:
texte_test = "Le patient reçoit BID une perfusion IV de soluté physiologique."
print(nettoyer_texte(texte_test)) 

Le patient reçoit bis in die une perfusion intraveineuse de soluté physiologique


# Exercice 3 : Normalisation Expert

### Objectifs :
1. Convertir certains termes médicaux (`"corticoïdes"` → `"glucocorticoides"`)
2. Traduire des expressions spécifiques (`"++fébrile"` → `"hyperthermique"`)
3. Corriger les erreurs de transcription (`"hyp0glycémie"` → `"hypoglycémie"`)

In [9]:
#Étape 1 : Dictionnaires de normalisation
termes_medicaux = {
    "corticoïdes": "glucocorticoides",
    "fébrile": "hyperthermique",
    "++fébrile": "hyperthermique",  # au cas où on détecte l'expression complète
}

corrections_orthographiques = {
    "hyp0glycémie": "hypoglycémie"
}

In [10]:
#Étape 2 : Fonction de correction orthographique
def corriger(mot):
    return corrections_orthographiques.get(mot, mot)

In [11]:
#Étape 4 : Fonction de normalisation
def normaliser_tokens(tokens):
    tokens_normalises = []
    for token in tokens:
        token = corriger(token)  # Correction orthographique
        if token in termes_medicaux:
            token = termes_medicaux[token]  # Normalisation médicale
        tokens_normalises.append(token)
    return tokens_normalises

In [12]:
# Test
tokens = tokeniser(texte_nettoye)
print("Normalisation expert:\n", normaliser_tokens(tokens))

Normalisation expert:
 ['patient', 'admis', 'le', 'pour', 'COVID-19', 'sévère', 'CRP', 'N5', '-', 'Fièvre', 'à', 'traitement', 'par', 'glucocorticoides', 'intraveineuse', 'azithromycine', 'Attention', 'allergie', 'à', 'la', 'pénicilline']


# Tests des fonctions avec des comptes-rendus générés par ChatGPT

In [13]:
## Test 1 : Rapport standard
texte_medical1 = """Patient 62 ans admis le 25/04/24 pour pancréatite aiguë.
Bilan: Amylase 1500U/L, CRP 85mg/L.
Ttt: jeûne + analgésiques IV. BID surveillance.
Note: Allergie aux AINS (écrit 'AINSs' dans le CR original)."""

In [14]:
#Nettoyage
print("Avant nettoyage:\n", texte_medical1)
texte_nettoye=nettoyer_texte(texte_medical1)
print("\nAprès nettoyage:\n", texte_nettoye)

Avant nettoyage:
 Patient 62 ans admis le 25/04/24 pour pancréatite aiguë.
Bilan: Amylase 1500U/L, CRP 85mg/L.
Ttt: jeûne + analgésiques IV. BID surveillance.
Note: Allergie aux AINS (écrit 'AINSs' dans le CR original).

Après nettoyage:
 Patient admis le pour pancréatite aiguë Bilan Amylase CRP traitement jeûne analgésiques intraveineuse bis in die surveillance Note Allergie aux AINS écrit AINSs dans le CR original


In [15]:
#tokenisation
tokens1 = tokeniser(texte_medical1)
tokens1

['Patient',
 '62',
 'ans',
 'admis',
 'le',
 '25/04/24',
 'pour',
 'pancréatite',
 'aiguë',
 '.',
 'Bilan',
 ':',
 'Amylase',
 '1500',
 'U',
 '/',
 'L',
 ',',
 'CRP',
 '85',
 'mg',
 '/',
 'L.',
 'Ttt',
 ':',
 'jeûne',
 '+',
 'analgésiques',
 'IV',
 '.',
 'BID',
 'surveillance',
 '.',
 'Note',
 ':',
 'Allergie',
 'aux',
 'AINS',
 '(',
 'écrit',
 "'AINSs",
 "'",
 'dans',
 'le',
 'CR',
 'original',
 ')',
 '.']

In [16]:
#Normalisation expert
print("Normalisation expert:\n", normaliser_tokens(tokens))

Normalisation expert:
 ['patient', 'admis', 'le', 'pour', 'COVID-19', 'sévère', 'CRP', 'N5', '-', 'Fièvre', 'à', 'traitement', 'par', 'glucocorticoides', 'intraveineuse', 'azithromycine', 'Attention', 'allergie', 'à', 'la', 'pénicilline']


In [17]:
## Test 2 : Urgence cardiologique
texte_medical2 = """Pat. 78ans,++douleur thoracique. ECG: sus-décalage ST.
Troponine: 15ng/mL (N<14).
Ttt: Aspirine 250mg + héparine 5000UI SC."""

In [18]:
#Nettoyage
print("Avant nettoyage:\n", texte_medical2)
texte_nettoye=nettoyer_texte(texte_medical2)
print("\nAprès nettoyage:\n", texte_nettoye)

Avant nettoyage:
 Pat. 78ans,++douleur thoracique. ECG: sus-décalage ST.
Troponine: 15ng/mL (N<14).
Ttt: Aspirine 250mg + héparine 5000UI SC.

Après nettoyage:
 patient douleur thoracique ECG sus-décalage ST Troponine N14 traitement Aspirine héparine SC


In [19]:
#tokenisation
tokens2 = tokeniser(texte_medical2)
tokens2

['Pat',
 '.',
 '78',
 'ans',
 ',',
 '++douleur',
 'thoracique',
 '.',
 'ECG',
 ':',
 'sus-décalage',
 'ST',
 '.',
 'Troponine',
 ':',
 '15',
 'ng',
 '/',
 'mL',
 '(',
 'N',
 '<',
 '14',
 ')',
 '.',
 'Ttt',
 ':',
 'Aspirine',
 '250',
 'mg',
 '+',
 'héparine',
 '5000',
 'UI',
 'SC',
 '.']

In [20]:
#Normalisation expert
print("Normalisation expert:\n", normaliser_tokens(tokens2))

Normalisation expert:
 ['Pat', '.', '78', 'ans', ',', '++douleur', 'thoracique', '.', 'ECG', ':', 'sus-décalage', 'ST', '.', 'Troponine', ':', '15', 'ng', '/', 'mL', '(', 'N', '<', '14', ')', '.', 'Ttt', ':', 'Aspirine', '250', 'mg', '+', 'héparine', '5000', 'UI', 'SC', '.']


In [21]:
## Test 3 : Scan de note manuscrite
texte_medical3 = """Pat. 45F - Tdm cérébrale: lésion hypo-dense 2cm.
Rx thorax: opacités ++ -> PCR COVID-19+
Médicaments: paracétamol 1g x3/j (écrit 'paracetamol')"""

In [22]:
#Nettoyage
print("Avant nettoyage:\n", texte_medical3)
texte_nettoye=nettoyer_texte(texte_medical3)
print("\nAprès nettoyage:\n", texte_nettoye)

Avant nettoyage:
 Pat. 45F - Tdm cérébrale: lésion hypo-dense 2cm.
Rx thorax: opacités ++ -> PCR COVID-19+
Médicaments: paracétamol 1g x3/j (écrit 'paracetamol')

Après nettoyage:
 patient - Tdm cérébrale lésion hypo-dense Rx thorax opacités - PCR COVID-19 Médicaments paracétamol x écrit paracetamol


In [23]:
#tokenisation
tokens3 = tokeniser(texte_medical3)
tokens3

['Pat',
 '.',
 '45',
 'F',
 '-',
 'Tdm',
 'cérébrale',
 ':',
 'lésion',
 'hypo-dense',
 '2',
 'cm',
 '.',
 'Rx',
 'thorax',
 ':',
 'opacités',
 '++',
 '-',
 '>',
 'PCR',
 'COVID-19+',
 'Médicaments',
 ':',
 'paracétamol',
 '1',
 'g',
 'x3/j',
 '(',
 'écrit',
 "'paracetamol",
 "'",
 ')']

In [24]:
#Normalisation expert
print("Normalisation expert:\n", normaliser_tokens(tokens3))

Normalisation expert:
 ['Pat', '.', '45', 'F', '-', 'Tdm', 'cérébrale', ':', 'lésion', 'hypo-dense', '2', 'cm', '.', 'Rx', 'thorax', ':', 'opacités', '++', '-', '>', 'PCR', 'COVID-19+', 'Médicaments', ':', 'paracétamol', '1', 'g', 'x3/j', '(', 'écrit', "'paracetamol", "'", ')']


In [25]:
## Test 4 : Cas pédiatrique
texte_medical4 = """Nouveau-né 3.2kg - ICTère cutanéo-muqueux.
Bilirubine totale: 350μmol/L (N<340).
Photothérapie IV++/24h.
Attention: risque d'érythème!"""

In [26]:
#Nettoyage
print("Avant nettoyage:\n", texte_medical4)
texte_nettoye=nettoyer_texte(texte_medical4)
print("\nAprès nettoyage:\n", texte_nettoye)

Avant nettoyage:
 Nouveau-né 3.2kg - ICTère cutanéo-muqueux.
Bilirubine totale: 350μmol/L (N<340).
Photothérapie IV++/24h.
Attention: risque d'érythème!

Après nettoyage:
 Nouveau-né - ICTère cutanéo-muqueux Bilirubine totale 350μmolL N340 Photothérapie intraveineuse Attention risque dérythème


In [27]:
#tokenisation
tokens4 = tokeniser(texte_medical4)
tokens4

['Nouveau-né',
 '3.2',
 'kg',
 '-',
 'ICTère',
 'cutanéo-muqueux',
 '.',
 'Bilirubine',
 'totale',
 ':',
 '350μmol',
 '/',
 'L',
 '(',
 'N',
 '<',
 '340',
 ')',
 '.',
 'Photothérapie',
 'IV++/24',
 'h',
 '.',
 'Attention',
 ':',
 'risque',
 "d'érythème",
 '!']

In [28]:
#Normalisation expert
print("Normalisation expert:\n", normaliser_tokens(tokens4))

Normalisation expert:
 ['Nouveau-né', '3.2', 'kg', '-', 'ICTère', 'cutanéo-muqueux', '.', 'Bilirubine', 'totale', ':', '350μmol', '/', 'L', '(', 'N', '<', '340', ')', '.', 'Photothérapie', 'IV++/24', 'h', '.', 'Attention', ':', 'risque', "d'érythème", '!']


In [29]:
## Test 5 : Oncologie complexe
texte_medical5 = """Pat. cancéreux sous FOLFIRINOX (dose:85mg/m2).
Complication: neutropénie grade4 (ANC<500).
Rx: pneumopathie -> Ttt ATB IV (amikacine 15mg/kg/J).
"""

In [30]:
#Nettoyage
print("Avant nettoyage:\n", texte_medical5)
texte_nettoye=nettoyer_texte(texte_medical5)
print("\nAprès nettoyage:\n", texte_nettoye)

Avant nettoyage:
 Pat. cancéreux sous FOLFIRINOX (dose:85mg/m2).
Complication: neutropénie grade4 (ANC<500).
Rx: pneumopathie -> Ttt ATB IV (amikacine 15mg/kg/J).


Après nettoyage:
 patient cancéreux sous FOLFIRINOX dose2 Complication neutropénie grade4 ANC500 Rx pneumopathie - traitement ATB intraveineuse amikacine


In [31]:
#tokenisation
tokens5 = tokeniser(texte_medical5)
tokens5

['Pat',
 '.',
 'cancéreux',
 'sous',
 'FOLFIRINOX',
 '(',
 'dose:85',
 'mg',
 '/',
 'm2',
 ')',
 '.',
 'Complication',
 ':',
 'neutropénie',
 'grade4',
 '(',
 'ANC',
 '<',
 '500',
 ')',
 '.',
 'Rx',
 ':',
 'pneumopathie',
 '-',
 '>',
 'Ttt',
 'ATB',
 'IV',
 '(',
 'amikacine',
 '15',
 'mg',
 '/',
 'kg/J',
 ')',
 '.']

In [32]:
#Normalisation expert
print("Normalisation expert:\n", normaliser_tokens(tokens5))

Normalisation expert:
 ['Pat', '.', 'cancéreux', 'sous', 'FOLFIRINOX', '(', 'dose:85', 'mg', '/', 'm2', ')', '.', 'Complication', ':', 'neutropénie', 'grade4', '(', 'ANC', '<', '500', ')', '.', 'Rx', ':', 'pneumopathie', '-', '>', 'Ttt', 'ATB', 'IV', '(', 'amikacine', '15', 'mg', '/', 'kg/J', ')', '.']
